In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp


### Read in TMAX Weather Data from us_ca_daily_weather_limited_TMAX.

- THis file has dropped all data without true TMAX values

In [2]:
# Read weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_df = pd.read_parquet('result_files/us_ca_daily_weather_limited_TMAX.parquet.gzip', engine="fastparquet") 

In [3]:
# Show weather data column types
weather_data_df.dtypes 

Attr
ID              object
Date    datetime64[ns]
PRCP           float64
TAVG           float64
TMAX           float64
TMIN           float64
dtype: object

In [4]:
# Show weather data
weather_data_df

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
index,,,,,,
0,CA001011500,2023-01-01,0.000000,41.00,46.40,35.60
1,CA001011500,2023-01-02,0.078740,41.00,44.60,37.40
2,CA001011500,2023-01-03,0.472441,39.74,41.90,37.40
3,CA001011500,2023-01-04,0.629921,40.64,43.70,37.40
4,CA001011500,2023-01-05,0.669291,44.60,48.20,41.00
...,...,...,...,...,...,...
8857244,USW00096409,2023-12-27,0.000000,-4.90,9.50,-19.30
8857245,USW00096409,2023-12-28,0.000000,0.14,5.90,-5.62
8857246,USW00096409,2023-12-29,0.000000,-7.69,-1.48,-13.90


### Determine how many days per year in given temperature ranges

#### Approach

Goal: How many days of year in each temperature range

- Create bins for highs (50-55, 55-6, …. 95-100, 100-105) etc
- Create bins for lows
- <or> Alternate: may also try bins for days > 80, days > 90, days < 30 etc.
- Save results

Result should look like:
| Station ID | #days < 10 | #days < 20 | ... | #days < 65 | #days > 60 | ... | #days > 95 | # days > 100 |
|----|----|----|----|----|----|----|----|----|
| US001 | 0 | 0 | ... | 300 | 365 | ... | 45 | 20 |
| US002 | 5 | 15 | ... | 360 | 280 | ... | 5 | 0 |

#### Find outliers in TMAX to refine the bins

In [5]:
# Template for binning TMAX values:
# ------------------------------------------
temp_bins = [-300, -200, -100, -50, 0, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
#temp_headings = ['-300 To -200', '-200 To -100', '-100 To -50', '-50 To 0', '0 To 50', '50 To 100', '100 To 200', '200 To 300', '300 To 400', '400 To 500', '500 To 600', '600 To 700', '700 To 800', '800 To 900', '900 to 1000']

### TMAX
weather_data_TMAX_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TMAX'], temp_bins)).rename_axis(columns=None, index=None)
weather_data_TMAX_df

,"(-100, -50]","(-50, 0]","(0, 50]","(50, 100]","(100, 200]","(200, 300]","(300, 400]","(400, 500]","(500, 600]","(600, 700]","(800, 900]"
CA001011500,0,0,130,213,0,0,0,0,0,0,0
CA001012055,0,0,79,133,2,0,0,0,0,0,0
CA001012475,0,0,128,236,0,0,0,0,0,0,0
CA001012710,0,0,123,229,0,0,0,0,0,0,0
CA001014820,0,0,160,202,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,163,148,0,0,0,0,0,0,0
USW00096406,0,30,206,128,0,0,0,0,0,0,0
USW00096407,0,27,176,115,0,0,0,0,0,0,0
USW00096408,0,13,239,112,0,0,0,0,0,0,0


In [6]:
# Fix Headings
temp_headings = ['-100 To -50', '-50 To 0', '0 To 50', '50 To 100', '100 To 200', '200 To 300', '300 To 400', '400 To 500', '500 To 600', '600 To 700', '800 To 900']

counter =-1
for h in temp_headings:
    counter += 1 
    weather_data_TMAX_df.rename(columns={weather_data_TMAX_df.columns[counter]: h}, inplace = True)

In [7]:
# Show Outliers
print((weather_data_TMAX_df != 0).sum())

-100 To -50       6
-50 To 0       1694
0 To 50        8314
50 To 100      8654
100 To 200     2949
200 To 300        2
300 To 400        1
400 To 500        1
500 To 600        1
600 To 700        3
800 To 900        2
dtype: int64


##### Outliers

x<-50 has 6 outliers<br>
x>200 has 10 outliers

#### Refine Bins

In [8]:
temp_bins = [ -500, -400, -300, -200, -100, -50, 0, 25, 50, 75, 100, 125, 150, 200, 250, 300, 350, 400, 500]
#temp_headings = ['-500 To -400', '-400 To -300', '-300 To -200', '-200 To -100', '-100 To -50', '-50 To 0', '0 To 25', '25 To 50', '50 To 75', '75 To 100', 
#                 '100 To 125', '125 To 150', '150 To 200', '200 To 250', '250 To 300', '300 To 350', '350 To 400, ', '400 To 500']

weather_data_TMAX_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TMAX'], temp_bins)).rename_axis(columns=None, index=None)

weather_data_TMAX_df

,"(-100, -50]","(-50, 0]","(0, 25]","(25, 50]","(50, 75]","(75, 100]","(100, 125]","(125, 150]","(150, 200]","(250, 300]","(300, 350]","(400, 500]"
CA001011500,0,0,0,130,160,53,0,0,0,0,0,0
CA001012055,0,0,0,79,75,58,2,0,0,0,0,0
CA001012475,0,0,0,128,226,10,0,0,0,0,0,0
CA001012710,0,0,0,123,214,15,0,0,0,0,0,0
CA001014820,0,0,0,160,144,58,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,3,160,147,1,0,0,0,0,0,0
USW00096406,0,30,107,99,110,18,0,0,0,0,0,0
USW00096407,0,27,80,96,105,10,0,0,0,0,0,0
USW00096408,0,13,120,119,107,5,0,0,0,0,0,0


In [9]:
# Fix Headings
temp_headings = ['-100 To -50', '-50 To 0', '0 To 25', '25 To 50', '50 To 75', '75 To 100', '100 To 125', '125 To 150', '150 To 200', '250 To 300', '300 To 350', '400 To 500']
counter =-1
for h in temp_headings:
    counter += 1 
    weather_data_TMAX_df.rename(columns={weather_data_TMAX_df.columns[counter]: h}, inplace = True)

In [10]:
# Show Outliers
print((weather_data_TMAX_df != 0).sum())

-100 To -50       6
-50 To 0       1694
0 To 25        5582
25 To 50       8310
50 To 75       8633
75 To 100      8423
100 To 125     2944
125 To 150        9
150 To 200        2
250 To 300        2
300 To 350        1
400 To 500        1
dtype: int64


##### Outliers

x<-50 has 6 outliers<br>
x>125 has 15 outliers

#### Define Final Bins

In [11]:
temp_bins = [-50, -25, -10, 0, 10, 20, 25, 32, 40, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 110, 120, 125]
#temp_headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 20', '20 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 95', '95 To 100', '100 To 110', '110 To 120', '120 To 125']

weather_data_TMAX_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TMAX'], temp_bins)).rename_axis(columns=None, index=None)
weather_data_TMAX_df

,"(-50, -25]","(-25, -10]","(-10, 0]","(0, 10]","(10, 20]","(20, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]",...,"(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 95]","(95, 100]","(100, 110]","(110, 120]","(120, 125]"
CA001011500,0,0,0,0,0,0,1,9,160,29,...,30,34,34,11,6,2,0,0,0,0
CA001012055,0,0,0,0,0,0,2,4,91,16,...,13,12,17,13,16,10,2,2,0,0
CA001012475,0,0,0,0,0,0,0,6,180,59,...,42,11,8,2,0,0,0,0,0,0
CA001012710,0,0,0,0,0,0,0,8,165,44,...,39,37,10,5,0,0,0,0,0,0
CA001014820,0,0,0,0,0,0,2,27,153,29,...,33,38,33,17,4,4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,0,0,0,3,13,63,133,50,...,9,4,1,0,0,0,0,0,0,0
USW00096406,2,11,17,40,49,18,42,28,49,21,...,25,23,10,6,2,0,0,0,0,0
USW00096407,0,6,21,30,31,19,43,28,52,20,...,23,16,9,1,0,0,0,0,0,0
USW00096408,0,1,12,29,54,37,31,43,68,23,...,21,11,5,0,0,0,0,0,0,0


In [12]:
# Fix Headings
temp_headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 20', '20 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 95', '95 To 100', '100 To 110', '110 To 120', '120 To 125']

counter =-1
for h in temp_headings:
    counter += 1 
    weather_data_TMAX_df.rename(columns={weather_data_TMAX_df.columns[counter]: h}, inplace = True)

In [13]:
# Sum all columns and rows TMAX
sums = weather_data_TMAX_df.sum().sum()
print(sums)

2957036


In [14]:
# Displaying the types of data
print("\nTypes of data:\n", weather_data_TMAX_df.dtypes)


Types of data:
 -50 To -25    int64
-25 To -10    int64
-10 To 0      int64
0 To 10       int64
10 To 20      int64
20 To 25      int64
25 To 32      int64
32 To 40      int64
40 To 55      int64
55 To 60      int64
60 To 65      int64
65 To 70      int64
70 To 75      int64
75 To 80      int64
80 To 85      int64
85 To 90      int64
90 To 95      int64
95 To 100     int64
100 To 110    int64
110 To 120    int64
120 To 125    int64
dtype: object


In [15]:
# Writing to parquet
weather_data_TMAX_df.to_parquet('result_files/us_ca_daily_TMAX_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

In [16]:
# Read TMIN weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_TMAX_df = pd.read_parquet('result_files/us_ca_daily_TMAX_weather.parquet.gzip', engine="fastparquet") 
weather_data_TMAX_df

,-50 To -25,-25 To -10,-10 To 0,0 To 10,10 To 20,20 To 25,25 To 32,32 To 40,40 To 55,55 To 60,...,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 95,95 To 100,100 To 110,110 To 120,120 To 125
index,,,,,,,,,,,,,,,,,,,,,
CA001011500,0,0,0,0,0,0,1,9,160,29,...,30,34,34,11,6,2,0,0,0,0
CA001012055,0,0,0,0,0,0,2,4,91,16,...,13,12,17,13,16,10,2,2,0,0
CA001012475,0,0,0,0,0,0,0,6,180,59,...,42,11,8,2,0,0,0,0,0,0
CA001012710,0,0,0,0,0,0,0,8,165,44,...,39,37,10,5,0,0,0,0,0,0
CA001014820,0,0,0,0,0,0,2,27,153,29,...,33,38,33,17,4,4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,0,0,0,3,13,63,133,50,...,9,4,1,0,0,0,0,0,0,0
USW00096406,2,11,17,40,49,18,42,28,49,21,...,25,23,10,6,2,0,0,0,0,0
USW00096407,0,6,21,30,31,19,43,28,52,20,...,23,16,9,1,0,0,0,0,0,0


### Repeat for TMIN

In [17]:
### TMIN 
temp_bins = [-50, -25, -10, 0, 10, 20, 25, 32, 40, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 110, 120]
#temp_headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 20', '20 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 95', '95 To 100', '100 To 110', '110 To 120']

weather_data_TMIN_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TMIN'], temp_bins)).rename_axis(columns=None, index=None)

weather_data_TMIN_df

,"(-50, -25]","(-25, -10]","(-10, 0]","(0, 10]","(10, 20]","(20, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 95]","(95, 100]","(100, 110]","(110, 120]"
CA001011500,0,0,0,0,0,2,12,95,166,48,15,2,1,0,1,1,0,0,0,0
CA001012055,0,0,0,0,0,1,20,37,69,38,18,5,3,5,4,8,4,1,1,0
CA001012475,0,0,0,0,0,0,4,41,299,20,0,0,0,0,0,0,0,0,0,0
CA001012710,0,0,0,0,0,0,7,84,217,44,0,0,0,0,0,0,0,0,0,0
CA001014820,0,0,0,0,2,4,19,126,141,40,21,9,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,2,11,27,28,70,57,116,0,0,0,0,0,0,0,0,0,0,0
USW00096406,42,40,37,34,43,19,29,24,86,10,0,0,0,0,0,0,0,0,0,0
USW00096407,23,31,25,31,38,17,30,29,73,21,0,0,0,0,0,0,0,0,0,0
USW00096408,2,38,46,54,48,24,26,42,83,1,0,0,0,0,0,0,0,0,0,0


In [18]:
# Sum all columns and rows TMIN
sums = weather_data_TMIN_df.sum().sum()
print(sums)

2956956


In [19]:
#Fix Headings
temp_headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 20', '20 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 95', '95 To 100', '100 To 110', '110 To 120']
counter =-1
for h in temp_headings:
    counter += 1 
    weather_data_TMIN_df.rename(columns={weather_data_TMIN_df.columns[counter]: h}, inplace = True)

In [20]:
# Write TMIN weather data to parquet file
# Writing to parquet
weather_data_TMIN_df.to_parquet('result_files/us_ca_daily_TMIN_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

# Displaying the types of data
print("\nTypes of data:\n", weather_data_TMIN_df.dtypes)


Types of data:
 -50 To -25    int64
-25 To -10    int64
-10 To 0      int64
0 To 10       int64
10 To 20      int64
20 To 25      int64
25 To 32      int64
32 To 40      int64
40 To 55      int64
55 To 60      int64
60 To 65      int64
65 To 70      int64
70 To 75      int64
75 To 80      int64
80 To 85      int64
85 To 90      int64
90 To 95      int64
95 To 100     int64
100 To 110    int64
110 To 120    int64
dtype: object


In [21]:
# Read TMIN weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_TMIN_df = pd.read_parquet('result_files/us_ca_daily_TMIN_weather.parquet.gzip', engine="fastparquet") 
weather_data_TMIN_df

,-50 To -25,-25 To -10,-10 To 0,0 To 10,10 To 20,20 To 25,25 To 32,32 To 40,40 To 55,55 To 60,60 To 65,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 95,95 To 100,100 To 110,110 To 120
index,,,,,,,,,,,,,,,,,,,,
CA001011500,0,0,0,0,0,2,12,95,166,48,15,2,1,0,1,1,0,0,0,0
CA001012055,0,0,0,0,0,1,20,37,69,38,18,5,3,5,4,8,4,1,1,0
CA001012475,0,0,0,0,0,0,4,41,299,20,0,0,0,0,0,0,0,0,0,0
CA001012710,0,0,0,0,0,0,7,84,217,44,0,0,0,0,0,0,0,0,0,0
CA001014820,0,0,0,0,2,4,19,126,141,40,21,9,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,2,11,27,28,70,57,116,0,0,0,0,0,0,0,0,0,0,0
USW00096406,42,40,37,34,43,19,29,24,86,10,0,0,0,0,0,0,0,0,0,0
USW00096407,23,31,25,31,38,17,30,29,73,21,0,0,0,0,0,0,0,0,0,0


### Repeat for TAvg

In [22]:
### TAVG
temp_bins = [-50, -25, -10, 0, 10, 20, 25, 32, 40, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 110, 120]
#temp_headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 20', '20 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 95', '95 To 100', '100 To 110', '110 To 120']

weather_data_TAVG_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TAVG'], temp_bins)).rename_axis(columns=None, index=None)

weather_data_TAVG_df

,"(-50, -25]","(-25, -10]","(-10, 0]","(0, 10]","(10, 20]","(20, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 95]","(95, 100]","(100, 110]","(110, 120]"
CA001011500,0,0,0,0,0,0,6,31,166,39,50,32,12,4,2,1,0,0,0,0
CA001012055,0,0,0,0,0,0,3,20,92,12,19,21,18,9,6,8,4,1,1,0
CA001012475,0,0,0,0,0,0,2,9,223,87,34,9,0,0,0,0,0,0,0,0
CA001012710,0,0,0,0,0,0,2,17,197,58,64,14,0,0,0,0,0,0,0,0
CA001014820,0,0,0,0,0,2,6,80,136,30,46,42,10,8,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,0,0,9,15,42,67,140,38,0,0,0,0,0,0,0,0,0,0
USW00096406,7,38,34,39,43,25,23,25,57,35,30,6,2,0,0,0,0,0,0,0
USW00096407,2,27,26,26,41,21,24,32,62,23,21,12,1,0,0,0,0,0,0,0
USW00096408,0,8,28,55,62,23,31,34,84,22,14,3,0,0,0,0,0,0,0,0


In [23]:
#Fix Headings
temp_headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 20', '20 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 95', '95 To 100', '100 To 110', '110 To 120']

counter =-1
for h in temp_headings:
    counter += 1 
    weather_data_TAVG_df.rename(columns={weather_data_TAVG_df.columns[counter]: h}, inplace = True)

In [24]:
# Sum all columns and rows TAVG
sums = weather_data_TAVG_df.sum().sum()
print(sums)

2957059


In [25]:
# Write TAVG weather data to parquet file

# Writing to parquet
weather_data_TAVG_df.to_parquet('result_files/us_ca_daily_TAVG_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

# Displaying the types of data
print("\nTypes of data:\n", weather_data_TAVG_df.dtypes)


Types of data:
 -50 To -25    int64
-25 To -10    int64
-10 To 0      int64
0 To 10       int64
10 To 20      int64
20 To 25      int64
25 To 32      int64
32 To 40      int64
40 To 55      int64
55 To 60      int64
60 To 65      int64
65 To 70      int64
70 To 75      int64
75 To 80      int64
80 To 85      int64
85 To 90      int64
90 To 95      int64
95 To 100     int64
100 To 110    int64
110 To 120    int64
dtype: object


In [26]:
# Read TAVG weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_TAVG_df = pd.read_parquet('result_files/us_ca_daily_TAVG_weather.parquet.gzip', engine="fastparquet") 
weather_data_TAVG_df

,-50 To -25,-25 To -10,-10 To 0,0 To 10,10 To 20,20 To 25,25 To 32,32 To 40,40 To 55,55 To 60,60 To 65,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 95,95 To 100,100 To 110,110 To 120
index,,,,,,,,,,,,,,,,,,,,
CA001011500,0,0,0,0,0,0,6,31,166,39,50,32,12,4,2,1,0,0,0,0
CA001012055,0,0,0,0,0,0,3,20,92,12,19,21,18,9,6,8,4,1,1,0
CA001012475,0,0,0,0,0,0,2,9,223,87,34,9,0,0,0,0,0,0,0,0
CA001012710,0,0,0,0,0,0,2,17,197,58,64,14,0,0,0,0,0,0,0,0
CA001014820,0,0,0,0,0,2,6,80,136,30,46,42,10,8,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,0,0,0,0,9,15,42,67,140,38,0,0,0,0,0,0,0,0,0,0
USW00096406,7,38,34,39,43,25,23,25,57,35,30,6,2,0,0,0,0,0,0,0
USW00096407,2,27,26,26,41,21,24,32,62,23,21,12,1,0,0,0,0,0,0,0
